In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

# 리뷰 자유 질문

프롬프트

In [22]:
# tool 사용 판단할 때 오류가 많이 남, 영어 프롬프트 사용 권장
PROMPT_TEMPLATE_kor = """
# 지시 사항
당신은 결혼 관련 업체에 대한 정보를 제공하는 AI 어시스턴트입니다. 사용자가 웨딩 관련 업체의 이름을 제공하면, 웹 검색 도구를 사용하여 그 업체의 후기와 평판을 조사하세요. 사용자의 질문에 정확하고 친절하게 답변하는 것이 목표입니다.

# 도구
- 어시스턴트는 다음 도구들을 사용할 수 있습니다:

{tools}

- 도구를 사용할 때는 다음 형식을 사용하세요:

생각: 도구를 사용할 필요가 있는가? 예
액션: 수행할 조치, [{tool_names}] 중 하나여야 합니다
액션 입력: 조치에 필요한 입력, 반드시 한국어로 작성되어야 하며 결혼과 관련되어야 합니다
관찰: 조치의 결과
...생각/액션/액션 입력/관찰은 여러 번 반복될 수 없습니다

- 인간에게 응답할 내용이 있거나 도구를 사용할 필요가 없을 때는 반드시 다음 형식을 사용하세요:

생각: 아니오
최종 답변: [여기에 응답을 작성하세요]

# 결과
조사한 정보를 다음 기준에 따라 요약하세요:

    전반적인 평판: 긍정적 대 부정적 고객 리뷰의 비율
    강점: 고객들이 자주 칭찬한 서비스나 특징
    약점: 고객들이 문제 제기한 사항이나 개선이 필요한 부분
    특징: 해당 업체만의 독특한 서비스나 혜택

요약된 정보를 사용자가 이해하기 쉽게 전달하세요. 필요한 경우, 결혼 준비와 관련한 추가 팁도 간단히 제공하세요.

# 중요 사항
- 항상 한국어로 응답하세요.
- 사용자가 웨딩홀, 결혼식장, 스튜디오, 메이크업샵 등의 결혼 관련 업체 이름을 명확하게 제공하지 않은 경우, 응답하지 말고 업체명을 알려달라고 요구하세요.

# 시작!

이전 대화 기록:
{chat_history}

새로운 입력: {input}

{agent_scratchpad}

"""


In [24]:
PROMPT_TEMPLATE_eg = """
# INSTRUCTION
You are an AI assistant providing information about wedding-related businesses. When a user provides the name of a wedding-related business, use web search tools to research reviews and reputations of that business. Your goal is to respond accurately and courteously to the user's queries.

# Tools 
- Assistant has access to the following tools:

{tools}

- To use a tool, please use the following format:

Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, MUST be written in Korean and be related to wedding
Observation: the result of the action
...the Thought/Action/Action Input/Observation cannot repeat several times

- When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

Thought: No
Final Answer: [your response here]

# RESULTS 
Summarize the information you researched according to the following criteria:

    Overall Reputation: Ratio of positive to negative customer reviews.
    Strengths: Services or features frequently praised by customers.
    Weaknesses: Issues raised by customers or areas needing improvement.
    Features: Unique services or benefits offered by the business.
    
Deliver the summarized information in a way that is easy for the user to understand. If necessary, briefly provide additional tips related to wedding preparation.

# IMPORTANT
- Always respond in Korean.
- If the user does not provide a clear name of a wedding-related business, such as a wedding hall, ceremony venue, studio, or makeup shop, do not respond but ask for the business name.

# START!

Previous conversation history:
{chat_history}

New input: {input}

{agent_scratchpad}

"""


In [26]:
import os
import readline
import json
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from langchain.tools.render import render_text_description
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.prompts import PromptTemplate


def custom_input(prompt):
    readline.set_pre_input_hook(lambda: readline.insert_text(""))
    readline.set_startup_hook(lambda: readline.insert_text(""))
    try:
        return input(prompt)
    finally:
        readline.set_pre_input_hook(None)
        readline.set_startup_hook(None)


def main():

    memory = ConversationBufferMemory(memory_key="chat_history", input_key="input") #  #

    tools = [TavilySearchResults(max_results=4)]


    prompt = PromptTemplate(
        input_variables=["agent_scratchpad", "chat_history", "input", "tools", "tool_names"],
        template=PROMPT_TEMPLATE_eg #프롬프트
        )
    
    prompt = prompt.partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
    )
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    llm_with_stop = llm.bind(stop=["\nObservation"])

    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
            "chat_history": lambda x: x["chat_history"]
        }
        | prompt
        | llm_with_stop
        | ReActSingleInputOutputParser()
    )
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True
    )

    while True:
        user_input = custom_input("당신: ")
        if user_input.lower() == "exit":
            print("어시스턴트: 안녕히 가세요!")
            break

        try:
            response = agent_executor.invoke({
                "input": user_input
            })
            print("어시스턴트: " + response["output"])

        except Exception as e:
            print(f"어시스턴트: 죄송합니다. 오류가 발생했습니다: {str(e)}")


if __name__ == "__main__":
    main()




> Entering new AgentExecutor chain...
Thought: Yes  
Action: tavily_search_results_json  
Action Input: 신라호텔 웨딩홀 후기  [{'url': 'https://m.blog.naver.com/dea_barbie/223433880439', 'content': '오늘은 우리나라에서 제일 명성 높은 웨딩홀로 알려진 [신라호텔웨딩] 다녀온 하객후기 남기려고 해요. 신라호텔웨딩은 실내 다이너스티홀 과 야외 영빈관 이 있어요. 오늘은 신라호텔 다이너스티홀웨딩 하객후기를 남기려고 합니다!'}, {'url': 'https://snaketeacher.tistory.com/5307', 'content': "'다이너스티 홀' 신라호텔에서 가장 큰 홀 800명 가까이 수용가능. 국내 웨딩홀 중 가장 높은 층고로, 답답한 느낌없이 탁 트인 느낌. 걸어가는데만 몇 분이 걸리는 수준의 역대급 버진로드. 최고의 플로리스트들이 총출동하는 규모의 생화 데코레이션. 웨딩케이크 신부대기실 클라쓰... 혼주&하객"}, {'url': 'https://m.blog.naver.com/aysl1793/222758276637', 'content': '신라호텔 웨딩홀. 오늘은 신라호텔 웨딩홀로 갑니다!! 신라호텔 웨딩홀은 영빈관, 다이너스티홀 요렇게 두 개가 있는데 영빈관은 야외에 있고, 다이너스티홀은 실내에 있어용. 신라호텔 주차장. 저희는 호텔 고객 전용 주차장을 이용했어요. 5성급 호텔이니 주차'}, {'url': 'https://m.blog.naver.com/cho_bibi/223360098097', 'content': '웨딩베뉴 리스트중 두번째로 다녀온 신라호텔 다이너스티홀. 지금까지 우리의 (내) 웨딩 베뉴 기준! • 5성급 호텔. • 단독 예식. • 어두운 홀. • 높은 층고. • 일자 버진로드. 아쉽게도 상담간 날 다이너스티홀은 행사중이라 못봤다ㅠㅠ. 가계약은 이번주 토'}]Thought: No  
Final 